# Maximum Gift Exchange Chaos

A model designed to explore the impact of different gift exchange rules (e.g. White Elephant, Green Grinch, etc.). The primary goal is to find the rules that lead to maximum chaos, as defined as maximum number of gift steals (possibly -- probably? -- within some time constraint).

## Gift Exchange Model

We can model a gift exchange as a game with an equal number of `G` `gifts` and `P` `players`. Players `value` each gift according to their personal preferences.

    
### Rule Variants
Some gift exchange variants use slightly different rules. These variants can be mixed and matched.

#### Additional end-game swapping
This variant adds extra swaps at the end of the game after `p1` makes their swap. If `p1` does swap with another player's gift, that player becomes active and repeats step 4, and so on until the active player elects not to swap or there are no more eligible gifts.

#### All gifts start unwrapped
In this variant, all gifts in the pool are unwrapped. This means that, instead of players learning their preferences as gifts are opened, all players have complete information about their gift preferences throughout the entire game.

#### Per-round stealing constraints
- A gift can only be stolen `St` times per round, regardless of who is doing the stealing. In other words, once a gift is stolen within a given round, it is ineligble for the remainder of that round.
- No more than `Sg` gifts can be stolen per round. Once `Sg` is reached, the round concludes and a new round begins by the next player becoming active in step 2.

#### Per-player stealing constraints
In this variant, there is also a cap on the number of times `Sp` each *player* can be stolen from. For example, once a given player `Sp_n` has been stolen from `Sp_n = Sp` times, the gift they hold at that point is now ineligible. 

#### No gift stealing constraint
In this variant, there is no cap on the number of times a given gift can be stolen. In other words, `S = infinity`.

### Gift Values
Players have personal preferences and value gifts accordingly. Each `player` has a (potentially) unique subjective `subj_value` for each `gift`. We can therefore model gift subjective value as a `G` by `P` matrix of values `subj_value_gp in range(100)`.

But, if we allow for the notion that some gifts are objectively better than others (almost all gifts will be valued higher than a spray-painted tamborine, for example), we can model `subj_value` as normally distributed variance around a common `obj_value` level. 

In [12]:
import numpy as np
import math
import pandas as pd
import scipy.stats as stats

## Inititial Conditions, Simulation Parameters, and Setup

### First, we set some initial conditions for our model

In [2]:
# game setup
player_count = 10
gift_count = player_count

# gift values setup
gift_value_scale = (0,1)
gift_obj_value_distr = {'mean': .5, 'stdev': .2}
gift_subj_value_stdev = .2

### Next, we create two classes to model the simulation's state

#### Gift Class

Each **gift** has the following attributes:
- `is_unwrapped`: a boolean marking whether the gift is open (`True`) or wrapped (`False`)
- `owned_by`: the id of the `Player` that currently owns the gift; otherwise `None`
- `brought_by`: the id of the `Player` that brought the gift to the game
- `steals`: a counter for the number of times the gift is stolen (including swaps)
- `obj_value`: the gift's objective value, set randomly within the `gift_value_scale` based on the `gift_obj_value_distr` parameter
- `is_eligible`: a boolean marking whether the gift can still be stolen; must be `False` if either `owned == False` or `steals < gift_steals_cap`

In [424]:
class Gift:    
    # MAYBE: add a weakref of the Gift instances so that get_available_gifts() doesn't need a global variable as an argument

    
    def __init__(self, id, value_scale, value_distr, wrapped):
        self.id = id
        self.is_unwrapped = not wrapped
        self.owned_by = None
        self.steals = 0
        self.is_eligible = True
        self.obj_value = Gift.set_obj_value(value_scale, value_distr)
        self.brought_by = None
    
    # helper function to create a truncated normal distribution
    @staticmethod
    def create_trunc_norm_distr(scale, mean, stdev):
        lower, upper = scale[0], scale[1]
        return stats.truncnorm(
            (lower - mean) / stdev,
            (upper - mean) / stdev,
            loc = mean,
            scale = stdev)
    
    # set an objective value for a gift
    @staticmethod
    def set_obj_value(scale, distr):
        mean, stdev = distr['mean'], distr['stdev']
        trunc_norm_distr = Gift.create_trunc_norm_distr(scale, mean, stdev)
        value_array = trunc_norm_distr.rvs(1)
        return value_array[0]
    
    @staticmethod
    def get_unowned_gifts(giftIDs):
        unowned_gifts = []
        
        for g in giftIDs:
            gift = gifts[g]
            if gift.owned_by is not None:
                unowned_gifts.append(g)
                
        return unowned_gifts
    
    @staticmethod
    def get_available_gifts(giftIDs):
        available_gifts = []
    
        for g in giftIDs:
            gift = gifts[g]
            if gift.is_eligible and (gift.owned_by is None):
                available_gifts.append(g)
                
            elif not gift.is_eligible:
                pass
                
            elif players[gift.owned_by].is_stealable:
                available_gifts.append(g)
                
        return available_gifts
    
# generate gifts 
gifts = {}

for g in range(gift_count):
    gifts[g] = Gift(g, gift_value_scale, gift_obj_value_distr, wrapped = True)

In [425]:
{key:value.obj_value for (key, value) in gifts.items()}

{0: 0.16733967987417658,
 1: 0.3652866375032492,
 2: 0.21973477816107384,
 3: 0.1925941555680286,
 4: 0.4268864882218387,
 5: 0.5721165178360115,
 6: 0.5697566837548489,
 7: 0.38934105060948704,
 8: 0.3497518304038306,
 9: 0.5187285045411646}

#### Player Class

Each **player** has the following attributes:
- `brought_gift`: the id of the `Gift` the player brought to the game; the player always knows their subjective value of this gift
- `owns_gift`: the id of the `Gift` the player currently owns; otherwise `None`
- `steals`: a counter for the number of times the player has a gift *stolen from them*
- `is_stealable`: a boolean marking whether the player can still be stolen *from*
- `is_active`: a boolean marking whether it's this player's turn to select, steal, or swap a gift
- `subj_values`: Subjective value for a given player&rightarrow;gift is set randomly around the gift's `obj_value` based on the `gift_subj_value_stdev` parameter

In [448]:
class Player:
    # MAYBE: add a weakref of the Player instances so that active_player() doesn't need a global variable as an argument
    
    def __init__(self, id, value_scale, subj_value_stdev, strategy, giftIDs):
        self.id = id
        self.brought_gift = id
        self.strategy = strategy
        self.steals = 0
        self.is_stealable = True
        self.owns_gift = None
        self.is_active = True if id == 0 else False # only the first player starts out as active
        self.subj_values = {}
        
        gifts[p].brought_by = p
        
        for gift in giftIDs:
            self.subj_values[gift] = self.init_gift_subj_value(gift_value_scale, 
                                                              gifts[gift].obj_value, 
                                                              gift_subj_value_stdev)
    
    # create a subjective value for a gift around its objective value
    # to be used in __init__
    def init_gift_subj_value(self, scale, obj_value, stdev):
        distr = Gift.create_trunc_norm_distr(scale, obj_value, stdev)
        value_array = distr.rvs(1)
        return value_array[0]
    
    def get_subj_value_of(self, giftIDs):
        subj_values = { key:value for key, value in self.subj_values.items() if key in giftIDs }
        #print('subjective_values: ', subj_values)
        return subj_values
    
    # get unwrapped gifts that self knows the value for, i.e. are either unowned or stealable or brought by self
    def available_known_gifts(self, giftIDs):
        available_known_gifts = Gift.get_available_gifts(giftIDs)
        brought = self.brought_gift
    
        if (brought not in available_known_gifts) and (gifts[brought].owner is not None):
            available_known_gifts.append(brought)
            
        return available_known_gifts
    
    # get list of tuples of available known gifts and their subjective values for self
    def sorted_available_known_gift_values(self, giftIDs):
        available_known_gifts = self.available_known_gifts(giftIDs)
        unsorted = self.get_subj_value_of(available_known_gifts)
        #print(unsorted)
        
        sorted_desc = dict(sorted([(value,key) for (key, value) in unsorted.items()], reverse = True))
        sorted_desc = { value:key for (key, value) in sorted_desc.items() } # make giftIDs dict keys again
        
        return sorted_desc

    
    # get self's best available known gift
    def best_available_known_gift_and_value(self, giftIDs):
        best_gift = list(self.sorted_available_known_gift_values(giftIDs).items())[0]
        #print(best_gift_id)
        return (best_gift[0], best_gift[1])
    
    @staticmethod
    def active_player(playerIDs):
        active = { key:value for key,value in players[playerIDs] }
    
# generate players

players = {}
for p in range(player_count):
    players[p] = Player(p, gift_value_scale, 
                          gift_subj_value_stdev, 
                          'steal_gift_above_threshold', 
                          gifts.keys())

In [ ]:
#gifts[5].is_eligible = False
#players[9].sorted_available_known_gift_values(gifts)
best = players[9].best_available_known_gift_and_value(gifts.keys())
print(best)
#print(players[9].subj_values)
#players[9].sorted_available_known_gift_values(gifts)

In [455]:
player_ids = players.keys()
print(player_ids)

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


## Policies



### Game Rules and Logic

#### Basic Rules
Each player brings one gift to the game and leaves with a (typically) different gift.

The game begins with all gifts wrapped, i.e. each player knows the content of only the gift they brought and none other.

A player order is determined (by some arbitrary method, e.g. random).

1. The first player `p1` selects one of the gifts from the set of unwrapped gifts and opens it.
2. The next player `p2` is now active, and has the option to either:
    1. Open a gift from the pool, leading to a repeat of step 2 with `p3` active
    2. Steal an eligible open gift, leading to a repeat of step 2 with `p1` active
3. Repeat step 2 until all gifts have been opened (i.e. there are no gifts remaining in the pool).
4. `p1` is now active again and has the option to (forcibly) swap with another player for their eligible gift. The game ends once `p1` has swapped or decided to not swap.

A new **round** begins when a new gift is opened from the pool.

##### Stealing
- Each gift can only be stolen up to `gift_steals_cap` times, including swaps. A given gift `G_n` is eligible for stealing (or swapping) while `gift_steals_n < gift_steals_cap`. For example, when `gift_steals_cap = 3`, each gift can only be stolen 3 times. After that, the gift is no longer eligible and no player may steal it or swap for it; a player holding an ineligible gift will leave the game with that gift.
- Within a given round, gifts cannot be stolen back by the player who began the turn holding that gift.

    1. If they do not swap, the game ends.
    2. If they do swap with another player, step 4 repeats with that swapped player active

In [154]:
# game rules, used as simulation parameters
game_params = {
    # GAME RULES
    'gift_steals_cap': 3, # if none, there is no cap
    'player_steals_cap':  None, # if none, there is no cap
    #'round_total_steals_cap': None, # number of total steals allowed per round
    #'round_gift_steals_cap': None, # number of steals allowed per round per gift
    'swapping': False, # does the game end with swaps (beginning with player 1)?
    'cascading_swaps': False, # False: only the first player is allowed to swap gifts at the end of the game; False: unlimited swaps are allowed
    'gifts_start_wrapped': True,
    
    # STRATEGY PARAMS
    'steal_threshold': gift_value_scale[1] * 2/3
}

## State Update Functions

In [ ]:
# player actions

# select a gift (and unwrap it if necessary); must be unowned
def select_gift(active_playerID, giftID):
    active_player = players[active_playerID]
    gift = gifts[giftID]
    
    if not active_player.is_active:
        raise Exception("It's not Player " + str(active_playerID) + "'s turn. Only the active player can make a move.")
        
    elif gift.owned_by is not None:
        raise Exception("Gift " + str(giftID) + " is already owned. Try stealing it instead.")
        
    else:
        gift.owned_by = active_playerID
        active_player.owns_gift = giftID
        
        if not gift.is_unwrapped
            gift.is_unwrapped = True
            
        active_player.is_active = False
        new_active_playerID = active_playerID + 1
        players[new_active_playerID].is_active = True
        
        print("Player " + str(active_playerID) + " selected Gift " + str(giftID))

# steal a gift; gift must be owned and eligible; current owner must be is_stealable.
def steal_gift(active_playerID, giftID):
    active_player = players[active_playerID]
    gift = gifts[giftID]
    target_player = players[gift.owned_by]
    
    if not active_player.is_active:
        raise Exception("It's not Player " + str(active_player) + "'s turn. Only the active player can make a move.")
    
    elif gift.owned_by is None:
        raise Exception("Gift " + str(giftID) + " is not yet owned. Try selecting it instead.")
        
    elif gift.is_eligible == False:
        raise Exception("Gift " + str(giftID) + " is ineligible and cannot be stolen.")
        
    elif target_player.is_stealable == False:
        raise Exception("Player " + str(target_player.id) + " cannot be stolen from.")
        
    else:
        gift.owned_by = active_playerID
        gift.steals += 1
        if gift.steals >= params['gift_steals_cap']:
            gift.is_eligible = False
        
        target_player.steals += 1
        if target_player.steals >= params['player_steals_cap']:
            target_player.is_stealable = False
        
        active_player.owns_gift = giftID
        target_player.owns_gift = None
        
        active_player.is_active = False
        target_player.is_active = True
        
        print("Player " + str(active_playerID) + " stole Gift " + str(giftID) + " from Player " + str(target_player.id))

# swap for a gift; all gifts must be owned; this gift must be eligible; current owner must be is_stealable
def swap_for_gift(active_playerID, target_giftID):
    active_player = players[active_playerID]
    current_giftID = active_player.owns_gift
    current_gift = gifts[current_giftID]
    target_gift = gifts[target_giftID]
    target_player = players[target_gift.owned_by]
    
    if not active_player.is_active:
        raise Exception("It's not Player " + str(active_player) + "'s turn. Only the active player can make a move.")
    
    elif target_gift.owned_by is None:
        raise Exception("Gift " + str(giftID) + " is not yet owned. Try selecting it instead.")
        
    elif gift.is_eligible == False:
        raise Exception("Gift " + str(giftID) + " is ineligible and cannot be stolen.")
        
    elif target_player.is_stealable == False:
        raise Exception("Player " + str(target_player.id) + " cannot be stolen from.")
        
    else:
        target_gift.owned_by = active_playerID
        current_gift.owned_by = target_playerID
        
        target_gift.steals += 1
        if target_gift.steals >= params['gift_steals_cap']:
            target_gift.is_eligible = False
        
        target_player.steals += 1
        if target_player.steals >= params['player_steals_cap']:
            target_player.is_stealable = False
        
        active_player.owns_gift = target_giftID
        target_player.owns_gift = current_giftID
        
        active_player.is_active = False
        if params['cascading_swaps']:
            target_player.is_active = True
            
        print("Player " + str(active_playerID) + " swapped gifts with Player " + str(target_player.id))

### Player Strategies

1. Steal second most valuable unwrapped gift
2. Steal most valuable unwrapped gift
3. Steal open gift if `subj_value` is greater than...
    1. Average of `subj_values` of the unwrapped gifts
    2. The maximum `subj_values` of the unwrapped gifts
    3. An arbitrary threshold, e.g. 80
4. Steal an open gift if about to become unstealable

Possible strategies (from https://github.com/BenCasselman/YankeeSwap):
1. Player steals with probability p = (number of gifts taken) / N (naive)
2. Player always steals most valuable gift available
3. Player always steals second-most-valuable gift available (if only one gift is available, player steals that one)
4. Player never steals
5. Player steals if any stealable gift has value (to them) greater than estimated underlying value of average gift
6. Player steals about-to-be unstealable gift if one available greater than estimated underlying value of avg gift
7. Same as #5 but factor in knowledge of gift player brought.
8. Ghosh-Mahdian: Player steals if best available gift has value > theta


To keep things simple at first, we'll use a single strategy for all players: 
- steal the highest `subj_value` open gift if `subj_value` >= 0.67

In [146]:
# player strategies

# in each of the following, "best" gift refers to the gift with the highest subjective value
strategies = [
    'steal_gift_above_threshold', # if the best unwrapped gift is owned / eligible, steal it if above steal_threshold
    'best_unwrapped_gift', # steal the best unwrapped gift if its owned / eligible, otherwise select it from pool
    'steal_second_best_gift', # if the best unwrapped gift is owned, steal the second best owned / eligible gift
    'steal_almost_ineligible_gift', # if stealing a gift would turn it ineligible, steal it if above steal_threshold
]

# if the best unwrapped gift is owned / eligible, steal it if above steal_threshold
def strategy_steal_gift_above_threshold(active_player, gifts, players, steal_threshold):
    gift_ids = gifts.keys()
    
    if best_value < steal_threshold:
        unowned = Gift.get_unowned_gifts(gift_ids)
        random_gift_id = np.random.choice(unowned)
        
        select_gift(active_player, random_gift_id)
        
    else:
        best_gift_id, best_value = active_player.best_available_known_gift_and_value(gift_ids)
        best_gift = gifts[best_gift_id]
        
        if best_gift.owned_by is None:
            # select best known unowned gift
            select_gift(active_player, best_gift_id)
    
        else:
            # steal the best gift
            steal_gift(active_player, best_gift_id)
            
    return (gifts, players)

In [ ]:
# turns and rounds

def player_turn(params, step, sL, s, _input):
    players = s['players']
    player_ids = players.keys()
    
    gifts = s['gifts']
    gift_ids = gifts.keys()
    
    active_player = Player.active_player(player_ids)
    
    # active player selects strategy
    #active_strategy = active_player.strategy
    #active_strategy_func = strategies[active_strategy]
    
    gifts, players = strategy_steal_gift_above_threshold(active_player, players, gifts, params['steal_threshold'])
    
    # make sure to increment subtimestep after each turn
    subtimestep += 1
    
    s['players'] = players
    s['gifts'] = gifts
    

def play_round(params, step, sL, s, _input):
    players = s['players']
    gifts = s['gifts']
    
    # return players and gifts
    pass

## Partial State Update Blocks

In [ ]:
partial_state_update_blocks = [
    { # complete a single round. A round ends when a player selects a gift from the pool. 
      # Within each round, multiple players can take a turn.
        'policies': {
            # Not going to use policies in this simulation, for the following reason:
            #  - Each player turn needs to react to state changes from the last player's turn.
            #  - But we don't know how many turns will exist in a given round.
            #  - Therefore, we can't separate out turns into partial state update blocks, which means using policies
            #    to set intentions for individual turns is not viable.
            #  - Instead, we'll set define strategies (which determin how they'll use their turn) as functions outside
            #    the partial state update block structure, and execute those strategies iteratively by player within
            #    the state update functions.
        },
        'variables': {
            # Each state update function per round will update state iteratively for each turn in that round,
            # and will increment the subtimestep variable directly after each turn.
            'players': ,
            'gifts': 
        }
    }
]

## Simulation Execution

In [ ]:
from cadCAD.configuration.utils import config_sim
simulation_parameters = config_sim({
    'T': player_count,
    'N': 1,
    'M': game_params
})

# The configurations above are then packaged into a `Configuration` object
config = Configuration(initial_state = initial_conditions, #dict containing variable names and initial values
                       partial_state_update_blocks = partial_state_update_blocks, #dict containing state update functions
                       sim_config = simulation_parameters #dict containing simulation parameters
                      )

# Run the simulations
exec_mode = ExecutionMode()
exec_context = ExecutionContext(exec_mode.single_proc)
executor = Executor(exec_context, [config]) # Pass the configuration object inside an array
raw_result, tensor = executor.execute() # The `execute()` method returns a tuple; its first elements contains the raw results

## Simulation Results

In [ ]:
df = pd.DataFrame(raw_result)